# Synthetic Optical Flow from Fused Lidar


In [1]:
import sys
sys.path.append('/opt/psegs')

import numpy as np

from psegs.exp.fused_lidar import FusedWorldCloudTableBase

## SemanticKITTI

In [2]:
from psegs.exp.semantic_kitti import SemanticKITTISDTable
class SemanticKITTIFusedWorldCloudTable(FusedWorldCloudTableBase):
    SRC_SD_TABLE = SemanticKITTISDTable
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        seg_rdd = cls.SRC_SD_TABLE.get_segment_datum_rdd(spark, segment_uri)
        
        # SemanticKITTI has no cuboids, so the Fuser algo simply concats the cloud points
        def iter_task_rows(iter_sds):
            from pyspark import Row
            from oarphpy.spark import RowAdapter
            for sd in iter_sds:
                if sd.point_cloud is not None:
                    pc = sd.point_cloud
                    task_id = "%s.%s" % (sd.uri.segment_id, pc.extra['semantic_kitti.scan_id'])
                    yield Row(
                        task_id=task_id,
                        point_clouds=[pc],
                        cuboids=[])
        
        task_rdd = seg_rdd.mapPartitions(iter_task_rows)
        return task_rdd
        


## KITTI-360

In [3]:
from psegs.datasets.kitti_360 import KITTI360SDTable
class KITTI360OurFusedClouds(KITTI360SDTable):
    INCLUDE_FISHEYES = False
    INCLUDE_FUSED_CLOUDS = False  # Use our own fused clouds

class KITTI360OurFusedWorldCloudTable(FusedWorldCloudTableBase):
    SRC_SD_TABLE = KITTI360OurFusedClouds
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        datum_df = cls.SRC_SD_TABLE.get_segment_datum_df(spark, segment_uri)
        datum_df.registerTempTable('datums')
        spark.sql("""
          CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
          SELECT 
              CONCAT(uri.segment_id, '.', uri.extra.`kitti-360.frame_id`) AS task_id,
              FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
              COLLECT_LIST(point_cloud) AS point_clouds
          FROM datums
          WHERE 
              uri.topic LIKE '%cuboid%' OR uri.topic LIKE '%lidar%'
          GROUP BY task_id
        """)
        tasks_df = spark.sql('SELECT * FROM culi_tasks_df')
        return tasks_df.rdd


## NuScenes

In [7]:
!pip3 install nuscenes-devkit==1.1.2
from psegs.datasets.nuscenes import NuscStampedDatumTableBase
class NuscFusedWorldCloudTable(FusedWorldCloudTableBase):
    SRC_SD_TABLE = NuscStampedDatumTableBase
    
    SPLITS = ['train_detect', 'train_track']
    
    @classmethod
    def _filter_ego_vehicle(cls, cloud_ego):
        cloud_ego = cloud_ego[np.where(  ~(
                        (cloud_ego[:, 0] <= 1.5) & (cloud_ego[:, 0] >= -1.5) &  # Nusc lidar +x is +right
                        (cloud_ego[:, 1] <= 2.5) & (cloud_ego[:, 0] >= -2.5) &  # Nusc lidar +y is +forward
                        (cloud_ego[:, 1] <= 1.5) & (cloud_ego[:, 0] >= -1.5)   # Nusc lidar +z is +up
        ))]
        return cloud_ego
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        datum_df = cls.SRC_SD_TABLE.get_segment_datum_df(spark, segment_uri)
        datum_df.registerTempTable('datums')
        
        if cls.SRC_SD_TABLE.LABELS_KEYFRAMES_ONLY:
            # For Nusc: group by nuscenes-sample-token WITH KEYFRAMES
            spark.sql("""
              CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
              SELECT 
                  uri.extra.`nuscenes-sample-token` AS task_id,
                  FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
                  COLLECT_LIST(point_cloud) AS point_clouds
              FROM datums
              WHERE 
                uri.extra.`nuscenes-is-keyframe` = 'True' AND (
                  uri.extra['nuscenes-label-channel'] is NULL OR 
                  uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
                ) AND (
                  uri.topic LIKE '%cuboid%' OR
                  uri.topic LIKE '%lidar%'
                )
              GROUP BY task_id
            """)
        else:
            # For Nusc: group by nuscenes-sample-token WITH ALL FRAMES
            spark.sql("""
              CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
              SELECT 
                  CONCAT(uri.segment_id, '.', uri.timestamp) AS task_id,
                  FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
                  COLLECT_LIST(point_cloud) AS point_clouds
              FROM datums
              WHERE 
                (
                  uri.extra['nuscenes-label-channel'] is NULL OR 
                  uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
                ) AND (
                  uri.topic LIKE '%cuboid%' OR
                  uri.topic LIKE '%lidar%'
                )
              GROUP BY task_id
              HAVING SIZE(cuboids) > 0 AND SIZE(pcs) > 0
            """)
        
        tasks_df = spark.sql('SELECT * FROM culi_tasks_df')
        return tasks_df.rdd


## Start Spark

In [5]:
from psegs.spark import NBSpark
spark = NBSpark.getOrCreate()

2021-02-12 05:28:17,823	oarph 93627 : Using source root /opt/psegs/psegs 
2021-02-12 05:28:17,824	oarph 93627 : Using source root /opt/psegs 
2021-02-12 05:28:17,871	oarph 93627 : Generating egg to /tmp/tmp7h2rh4l0_oarphpy_eggbuild ...
2021-02-12 05:28:18,007	oarph 93627 : ... done.  Egg at /tmp/tmp7h2rh4l0_oarphpy_eggbuild/psegs-0.0.0-py3.8.egg


## Build Fused Lidar Assets

```
docker --context default run -it --name=potree_viewer --rm --net=host -v `pwd`:/shared  jonazpiazu/potree
```

In [8]:
seg_uris = NuscFusedWorldCloudTable.get_all_segment_uris()
sd_rdd = NuscFusedWorldCloudTable._get_segment_datum_rdd_or_df(spark, seg_uris[0])
print(sd_rdd.count())

2021-02-12 05:30:19,540	ps   93627 : Filtering to only 1 segments
2021-02-12 05:30:19,541	ps   93627 : NuscFusedWorldCloudTable building fused world clouds ...
2021-02-12 05:30:19,541	ps   93627 : ... have 1 segments to fuse ...
2021-02-12 05:30:19,542	ps   93627 : ... working on scene-1013 ...
2021-02-12 05:30:21,028	ps   93627 : Filtering to only 1 segments
2021-02-12 05:32:48,293	ps   93627 : ... computed world cloud for scene-1013 of shape (1063496, 3) (0.03 GB) ...
2021-02-12 05:32:48,293	ps   93627 : ... writing ply to /opt/psegs/dataroot/fused_world_clouds/naive_cuboid_scrubber/nuscenes/train_detect/scene-1013/fused_world.ply ...
2021-02-12 05:32:51,427	ps   93627 : ... done writing ply ...
INFO - 2021-02-12 05:32:51,427 - fused_lidar - ... done writing ply ...
2021-02-12 05:32:51,429	ps   93627 : ... NuscFusedWorldCloudTable done fusing clouds.
INFO - 2021-02-12 05:32:51,429 - fused_lidar - ... NuscFusedWorldCloudTable done fusing clouds.


1
